In [1]:
# Import Required Packages
import torch
import os
import sys
import re
import random
import importlib.util
from typing import *
from tqdm import tqdm 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

from JS_functions import *
from JS_Augmentaters import *
from seeds import *

common_path = "/home/jdelinea/ARC-AGI/BARC/seeds/common.py"
spec = importlib.util.spec_from_file_location("common", common_path)
common = importlib.util.module_from_spec(spec)
spec.loader.exec_module(common)

# Add the 'seeds' directory to sys.path
seeds_path = os.path.abspath("seeds")
if seeds_path not in sys.path:
    sys.path.append(seeds_path)

- The generating input is stochastic and might not produce a correct input. 
- At least 6 have a conception error ()

In [2]:
# Create instances of augmenters
list_group_action = [
    Rotate(90),
    Rotate(180),
    Flip(axis=0),
    Flip(axis=1),
    PermuteColors(),
    Dropout(),
]

# Create a dictionary with descriptive names as keys and augmenter instances as values
group_action_dict = {
    "Rotate 90°": Rotate(90),
    "Rotate 180°": Rotate(180),
    "Flip Vertical": Flip(axis=0),
    "Flip Horizontal": Flip(axis=1),
    "Permute Colors": PermuteColors(),
    "Dropout": Dropout(),
}

In [3]:
def invariance_seeds(module, module_name, max_attempts):
    if hasattr(module, "generate_input") and hasattr(module, "main"):
        #print(f"Running {module_name}...")

        for attempt in range(max_attempts):
            try:
                # Call the 'generate_input' function
                task_input = module.generate_input()
                #print(type(task_input))

                # Call the 'main' function with the generated input
                result = module.main(task_input)

                # If no exception occurs, verify stable for this transformation
                    
                break

            except Exception as e:
                #print(f"Attempt {attempt + 1}: Error - {e}")
                a = 1

        else:
            print(f"No successful execution found for {module_name} after {max_attempts} attempts.\n")

    else:
        print(f"{module_name} does not contain 'generate_input' or 'main' functions.\n")


In [4]:
def load_and_execute_until_success(folder, max_attempts=10):
    for filename in os.listdir(folder):
        if filename.endswith(".py") and filename not in ["__init__.py", "common.py", "25d487eb.py", "aabf363d.py","3e980e27.py","264363fd.py","f8a8fe49.py","6b9890af.py"]:
            module_name = filename[:-3]
            file_path = os.path.join(folder, filename)

            try:
                # Dynamically load the module
                spec = importlib.util.spec_from_file_location(module_name, file_path)
                module = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(module)  # Execute the module here

                # Call the invariance_seeds function
                invariance_seeds(module, module_name, max_attempts)
                
            except Exception as e:
                print(f"Error loading {module_name}: {e}\n")

In [5]:
# Specify the path to the 'seeds' folder
seeds_folder = "seeds"

# Execute the functions in all modules within the 'seeds' folder
load_and_execute_until_success(seeds_folder)


Seeds who did not produce correct inputs after 100 tries:

- 6b9890af : only length-1 arrays can be converted to Python scalars
- f8a8fe49 : 'int' object does not support item assignment
- 264363fd : 'NoneType' object has no attribute 'shape'
- 3e980e27 : local variable 'mask_color' referenced before assignment
- aabf363d : can only concatenate str (not "float") to str
- 25d487eb : index x is out of bounds for axis 1 with size x


In [6]:
def load_and_execute_until_success(folder,group_action_dict, max_attempts=10):
    for filename in os.listdir(folder):
        if filename.endswith(".py") and filename not in ["__init__.py", "common.py"]:
            module_name = filename[:-3]
            file_path = os.path.join(folder, filename)

            try:
                # Dynamically load the module
                spec = importlib.util.spec_from_file_location(module_name, file_path)
                module = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(module)  # Execute the module here

                # Call the invariance_seeds function
                invariance_seeds(module, module_name,group_action_dict, max_attempts)
                
            except Exception as e:
                print(f"Error loading {module_name}: {e}\n")

In [7]:
def invariance_seeds(module, module_name, group_action_dict, max_attempts):
    invariance_results = []  # List to store invariance information for each attempt

    if hasattr(module, "generate_input") and hasattr(module, "main"):
        print(f"Running {module_name}...")

        for attempt in range(max_attempts):
            try:
                # Call the 'generate_input' function
                task_input = module.generate_input()
                # Call the 'main' function with the generated input
                result = module.main(task_input)

                # Dictionary to store invariance for this attempt
                attempt_invariance = {
                    "attempt": attempt + 1,
                    "input": task_input,
                    "result": result,
                    "invariance": {}
                }

                # Check each transformation in group_action_dict
                for action_name, group_action in group_action_dict.items():
                    try:
                        # Apply the transformation to the input
                        transformed_input = group_action(task_input)
                        # Apply the main function to the transformed input
                        transformed_result = module.main(transformed_input)

                        # Check if the transformation maintains invariance
                        if transformed_result == group_action(result):
                            attempt_invariance["invariance"][action_name] = True
                        else:
                            attempt_invariance["invariance"][action_name] = False
                    except Exception as e:
                        attempt_invariance["invariance"][action_name] = f"Error: {e}"

                # Add the invariance results for this attempt to the list
                invariance_results.append(attempt_invariance)

            except Exception as e:
                print(f"Attempt {attempt + 1}: Error - {e}")

        # Print summary of invariance results
        for result in invariance_results:
            print(f"\nAttempt {result['attempt']}:")
            print(f"Input: {result['input']}")
            print(f"Result: {result['result']}")
            for action, is_invariant in result['invariance'].items():
                print(f"  {action}: {'Invariant' if is_invariant is True else is_invariant}")

    else:
        print(f"{module_name} does not contain 'generate_input' or 'main' functions.\n")

    return invariance_results


In [ ]:
# Specify the path to the 'seeds' folder
seeds_folder = "seeds"

# Execute the functions in all modules within the 'seeds' folder
load_and_execute_until_success(seeds_folder, group_action_dict)


Running 3de23699...

Attempt 1:
Input: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [5 0 0 0 0 0 0 0 0 0 0 5 0 0]
 [0 0 4 4 4 4 0 4 0 0 0 0 0 0]
 [0 0 4 4 4 4 0 4 4 4 4 0 0 0]
 [0 0 4 4 0 4 4 4 4 4 4 0 0 0]
 [0 4 4 4 4 4 4 4 4 4 4 0 0 0]
 [0 4 4 4 0 0 0 0 4 0 0 0 0 0]
 [0 4 4 4 0 0 0 0 4 0 0 0 0 0]
 [0 4 4 4 4 4 4 4 4 4 4 0 0 0]
 [0 0 4 4 0 4 4 4 4 4 4 0 0 0]
 [0 0 4 4 4 4 0 4 4 4 4 0 0 0]
 [0 0 4 4 4 4 0 4 0 0 0 0 0 0]
 [5 0 0 0 0 0 0 0 0 0 0 5 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Result: [[0 5 5 5 5 0 5 0 0 0]
 [0 5 5 5 5 0 5 5 5 5]
 [0 5 5 0 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5]
 [5 5 5 0 0 0 0 5 0 0]
 [5 5 5 0 0 0 0 5 0 0]
 [5 5 5 5 5 5 5 5 5 5]
 [0 5 5 0 5 5 5 5 5 5]
 [0 5 5 5 5 0 5 5 5 5]
 [0 5 5 5 5 0 5 0 0 0]]
  Rotate 90°: Error: The truth value of an array with more than one el

In [ ]:

grid = [
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4],
        [5, 5, 5, 5, 5],
    ]

print("Original Grid:")
for row in grid:
    print(row)

    # Create instances of augmenters
augmenters = [
    Rotate(90),
    Rotate(180),
    Flip(axis=0),
    Flip(axis=1),
    PermuteColors(),
    Dropout(),
    ]

# Apply each augmenter and print the result
for augmenter in augmenters:
    print(f"\nApplying {augmenter}:")
    augmented_grid = augmenter(grid)
    for row in augmented_grid:
        print(row)
